In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam    # 추가
from tensorflow.keras.preprocessing.image import ImageDataGenerator # 추가

#######################################
# 32x32 픽셀의 6만개 컬러이미지 포함
# 각 이미지는 10개의 클래스로 라벨링
# 50000개 이미지는 트레이닝 용도로 사용
# 10000개 이미지는 테스트 용도로 사용
#######################################

# 이전 CNN2.ipynb 실습의 70% 보다 확률을 높이는 작업 #

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

  # 255.0 정규화
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)


# 보강할 데이터 변형방식 결정
    # 기존 이미지를 활용하여 변형을 해서 새로운 이미지를 생성
    # (보강 = 확률을 높이기 위해, 기존 CNN2.ipynb에서는 70% 내외)

    
    # 기존사진 변형작업
gen = ImageDataGenerator(rotation_range=20, shear_range=0.2, width_shift_range=0.2,
                         height_shift_range=0.2, horizontal_flip=True) 

    
augment_ratio = 1.5 # 전체 데이터의 150%
augment_size = int(augment_ratio * x_train.shape[0])

randidx = np.random.randint(x_train.shape[0], size=augment_size)

x_augmented = x_train[randidx].copy()
y_augmented = y_train[randidx].copy()

x_augmented, y_augmented = gen.flow(x_augmented, y_augmented,
                                    batch_size=augment_size, shuffle=False).next()  # 보강할 이미지 생성

x_train = np.concatenate((x_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

s = np.arange(x_train.shape[0]) # 보강된 학습데이터, 정답데이터를 랜덤하게 섞는다
np.random.shuffle(s)

x_train = x_train[s]
y_train = y_train[s]

#

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print("Train samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(x_train[i])
  plt.xlabel(class_names[y_train[i][0]])
plt.show()

In [ ]:
# CNN 모델구축
cnn = Sequential()

cnn.add(Conv2D(32,(3,3), activation='relu', padding='same', input_shape=(32,32,3))) # cifar10 텐서(높이,너비,채널)
cnn.add(Conv2D(32,(3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(64,(3,3), activation='relu', padding='same')) 
cnn.add(Conv2D(64,(3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128,(3,3), activation='relu', padding='same')) 
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128,(3,3), activation='relu', padding='same')) 
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(256,(3,3), activation='relu', padding='same')) 
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten())  # 3차원 텐서를 1차원 벡터로 변환

cnn.add(Dense(128, activation='relu'))  # 은닉층 개념
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation='softmax'))    # 출력층




In [ ]:
# CNN 모델 컴파일 및 학습

cnn.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

hist = cnn.fit(x_train, y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test))


In [ ]:
cnn.evaluate(x_test, y_test)    # 모델 정확도 평가
    # 기존에는 70% 정도였지만 86% 내외로 올라갔음

In [ ]:
# 정확도 및 손실 1
import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy Trend')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='best')

plt.grid()
plt.show()

In [ ]:
# 정확도 및 손실 2
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss Trend')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='best')

plt.grid()
plt.show()